In [ ]:
!pip install /kaggle/input/piidd-installation-package/transformers-4.38.1-py3-none-any.whl -q
!pip install /kaggle/input/piidd-installation-package/accelerate-0.27.2-py3-none-any.whl -q
!pip install /kaggle/input/piidd-installation-package/bitsandbytes-0.42.0-py3-none-any.whl -q

In [ ]:
import json
import re
import argparse
import pandas as pd
import numpy as np
import time
import torch
from itertools import chain
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    BitsAndBytesConfig,
    pipeline
)
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from tqdm import tqdm

In [ ]:
INFERENCE_MAX_LENGTH = 1024
INFERENCE_STRIDE = 256
TRAINING_MODEL_PATH = ["/kaggle/input/pii-data-detection-models/checkpoint-900"]

In [ ]:
id2label = {
    0: "B-EMAIL",
    1: "B-ID_NUM",
    2: "B-NAME_STUDENT",
    3: "B-PHONE_NUM",
    4: "B-STREET_ADDRESS",
    5: "B-URL_PERSONAL",
    6: "B-USERNAME",
    7: "I-ID_NUM",
    8: "I-NAME_STUDENT",
    9: "I-PHONE_NUM",
    10: "I-STREET_ADDRESS",
    11: "I-URL_PERSONAL",
    12: "O"
}

label2id = {
    "B-EMAIL": 0,
    "B-ID_NUM": 1,
    "B-NAME_STUDENT": 2,
    "B-PHONE_NUM": 3,
    "B-STREET_ADDRESS": 4,
    "B-URL_PERSONAL": 5,
    "B-USERNAME": 6,
    "I-ID_NUM": 7,
    "I-NAME_STUDENT": 8,
    "I-PHONE_NUM": 9,
    "I-STREET_ADDRESS": 10,
    "I-URL_PERSONAL": 11,
    "O": 12
}

all_labels = [
    "B-EMAIL",
    "B-ID_NUM",
    "B-NAME_STUDENT",
    "B-PHONE_NUM",
    "B-STREET_ADDRESS",
    "B-URL_PERSONAL",
    "B-USERNAME",
    "I-ID_NUM",
    "I-NAME_STUDENT",
    "I-PHONE_NUM",
    "I-STREET_ADDRESS",
    "I-URL_PERSONAL",
    "O"
]

In [ ]:
df = pd.read_json("/kaggle/input/pii-detection-removal-from-educational-data/test.json")


def get_labels(word_ids, word_labels):
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        else:
            label_ids.append(label2id[word_labels[word_idx]])
    return label_ids

# Tokenize texts, possibly generating more than one tokenized sample for each text


def tokenize(df, to_tensor=True, with_labels=True):

    # This is what"s different from a longformer
    # Read the parameters with attention
    encoded = tokenizer(
        df["tokens"].tolist(),
        is_split_into_words=True,
        return_overflowing_tokens=True,
        stride=INFERENCE_STRIDE,
        max_length=INFERENCE_MAX_LENGTH,
        padding="max_length",
        truncation=True
    )

    if with_labels:
        encoded["labels"] = []

    encoded["wids"] = []
    n = len(encoded["overflow_to_sample_mapping"])
    for i in range(n):

        # Map back to original row
        text_idx = encoded["overflow_to_sample_mapping"][i]

        # Get word indexes (this is a global index that takes into consideration the chunking :D )
        word_ids = encoded.word_ids(i)

        if with_labels:
            # Get word labels of the full un-chunked text
            word_labels = df["labels"].iloc[text_idx]

            # Get the labels associated with the word indexes
            label_ids = get_labels(word_ids, word_labels)
            encoded["labels"].append(label_ids)

        encoded["wids"].append([w if w is not None else -1 for w in word_ids])

    if to_tensor:
        encoded = {key: torch.as_tensor(val) for key, val in encoded.items()}
    return encoded

In [ ]:
class PIIDataset(Dataset):
    def __init__(self, tokenized_ds):
        self.data = tokenized_ds

    def __getitem__(self, index):
        item = {k: self.data[k][index] for k in self.data.keys()}
        return item

    def __len__(self):
        return len(self.data["input_ids"])

In [ ]:
def inferenceV4(df, dl):

    # These 2 dictionaries will hold text-level data
    # Helping in the merging process by accumulating data
    # Through all the chunks

    token_pred = defaultdict(lambda: defaultdict(int))
    token_cnt = defaultdict(lambda: defaultdict(int))

    for batch in tqdm(dl):
        ids = batch["input_ids"].to("cuda")
        mask = batch["attention_mask"].to("cuda")
        preds = model(ids, attention_mask=mask, return_dict=False)[0].cpu().detach().numpy()
        preds_softmax = np.exp(preds) / np.sum(np.exp(preds), axis=2).reshape(preds.shape[0], preds.shape[1], 1)

        del ids, mask

        # Go over each prediction, getting the text_id reference

        for k, (chunk_preds, text_id) in enumerate(zip(preds_softmax, batch["overflow_to_sample_mapping"].tolist())):
            # The word_ids are absolute references in the original text
            word_ids = batch["wids"][k].numpy()

            for idx, word_idx in enumerate(word_ids):
                if word_idx != -1:
                    token_pred[text_id][word_idx] += chunk_preds[idx]
                    token_cnt[text_id][word_idx] += 1

    for text_id in token_pred:
        for word_idx in token_pred[text_id]:
            token_pred[text_id][word_idx] /= token_cnt[text_id][word_idx]

    return token_pred

In [ ]:
final_token_pred = defaultdict(lambda: defaultdict(int))
for model_path in TRAINING_MODEL_PATH:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenized_test = tokenize(df, with_labels=False)

    test_dataset = PIIDataset(tokenized_test)
    test_dataloader = DataLoader(test_dataset, batch_size=1)
    model = AutoModelForTokenClassification.from_pretrained(
        model_path,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )
    model.cuda()
    token_pred = inferenceV4(df, test_dataloader)
    for text_id in token_pred:
        for word_idx in token_pred[text_id]:
            final_token_pred[text_id][word_idx] += token_pred[text_id][word_idx] / len(TRAINING_MODEL_PATH)

In [ ]:
document, token, label = [], [], []
for text_id in final_token_pred:
    for word_idx in final_token_pred[text_id]:
        pred = final_token_pred[text_id][word_idx].argmax(-1)
        pred_without_O = final_token_pred[text_id][word_idx][:12].argmax(-1)
        if final_token_pred[text_id][word_idx][12] < 0.55:
            final_pred = pred_without_O
        else:
            final_pred = pred
        if id2label[final_pred] != "O":
            document.append(df.loc[text_id, "document"])
            token.append(word_idx)
            label.append(id2label[final_pred])

pred_df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label
})

In [ ]:
def score(row):
    doc = row.document
    tok = row.token
    doc_idx = df.query("document == @doc").index[0]
    return token_pred[doc_idx][tok][label2id[row.label]]

In [ ]:
pred_df["score"] = pred_df.apply(lambda x: score(x), axis=1)

## Post-Process 

In [ ]:
df = df[["document", "tokens"]].copy()

In [ ]:
df = df.explode(["tokens"]).reset_index(drop=True).rename(columns={"tokens": "token"})

In [ ]:
df["token_str"] = df["token"]
df["token"] = df.groupby("document").cumcount()

In [ ]:
new_pred_df = pd.merge(df, pred_df[["document", "token", "label", "score"]], on=["document", "token"], how="left")
new_pred_df["label"] = new_pred_df["label"].fillna("O")

In [ ]:
def pp(new_pred_df):
    df = new_pred_df.copy()
    i = 0
    while i < len(df):
        st = i
        doc = df.loc[st, "document"]
        tok = df.loc[st, "token"]
        pred_tok = df.loc[st, "label"]
        if pred_tok == "O":
            i += 1
            continue
        lab = pred_tok.split("-")[1]
        cur_doc = doc
        cur_lab = lab
        last_tok = tok
        cur_tok = last_tok

        while i < len(df) and cur_doc == doc and cur_lab == lab and last_tok == cur_tok:
            last_tok = cur_tok + 1
            i += 1
            cur_doc = df.loc[i, "document"]
            cur_tok = df.loc[i, "token"]
            if i >= len(df) or df.loc[i, "label"] == "O":
                break
            cur_lab = df.loc[i, "label"].split("-")[1]

        # exception
        if st - 2 >= 0 and df.loc[st - 2, "document"] == df.loc[st, "document"] and df.loc[st - 1, "token_str"] == "\n" and df.loc[st - 2, "label"] != "O" and df.loc[st - 2, "label"].split("-")[1] == lab:
            df.loc[st - 1, "label"] = "I-" + lab
            df.loc[st - 1, "score"] = 1
            for j in range(st, i):
                if df.loc[j, "label"] != "I-" + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = "I-" + lab
            continue

        # fix
        for j in range(st, i):
            if j == st:
                if df.loc[j, "label"] != "B-" + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = "B-" + lab
            else:
                if df.loc[j, "label"] != "I-" + lab:
                    df.loc[j, "score"] = 1
                    df.loc[j, "label"] = "I-" + lab

        if lab == "NAME_STUDENT" and any(len(item) == 2 and item[0].isupper() and item[1] == "." for item in df.loc[st:i - 1, "token_str"]):
            for j in range(st, i):
                df.loc[j, "score"] = 0
                df.loc[j, "label"] = "O"

    return df

In [ ]:
new_pred_df = pp(new_pred_df)

In [ ]:
new_pred_df

In [ ]:
new_pred_df = new_pred_df.query("label != 'O'").reset_index(drop=True)

In [ ]:
rows_to_delete = []
for idx, row in new_pred_df.iterrows():
    if row.label == "I-PHONE_NUM":
        if row.token_str == ")":
            rows_to_delete.append(idx)
        elif not bool(re.search(r"\d", row.token_str)):
            rows_to_delete.append(idx)
    elif row.label == "B-EMAIL":
        if "@" not in row.token_str:
            rows_to_delete.append(idx)

In [ ]:
new_pred_df = new_pred_df.drop(rows_to_delete, axis=0, inplace=False)

In [ ]:
new_pred_df["row_id"] = list(range(len(new_pred_df)))

In [ ]:
new_pred_df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)

In [ ]:
new_pred_df[["row_id", "document", "token", "label", "token_str"]]

In [ ]:
new_pred_df[["row_id", "document", "token", "label", "token_str"]].to_csv("output.csv", index=False)

## LLM Post-Process

In [ ]:
bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=torch.float16
    load_in_8bit=True
)

In [ ]:
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
# model_name = "HanNayeoniee/LHK_DPO_v1"
# model_name = "openchat/openchat-3.5-0106"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
df = new_pred_df[["row_id", "document", "token", "label", "token_str"]]

In [ ]:
unique_documents = df["document"].unique()
unique_documents

In [ ]:
new_df = pd.DataFrame(columns=["document", "token", "label"])
new_df.head()

In [ ]:
test = pd.read_json("/kaggle/input/pii-detection-removal-from-educational-data/test.json")

In [ ]:
def get_response(name, document, test):
    input_str = f"I'll give you a name, and you need to tell me if it's a normal person name, cited name or even not a name. Do not consider other factors.\nExample:\n- Is Matt Johnson a normal person name? Answer: Yes\n- Is Johnson. T a normal person name? Answer: No, this is likely a cited name.\n- Is Andsgjdu a normal person name? Answer: No, it is even not a name.\nNow the question is:\n- Is {name} a normal person name? Answer:"

    sequences = pipe(
        input_str,
        do_sample=True,
        max_new_tokens=10,
        temperature=0.01,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    generated_text = sequences[0]["generated_text"]
    answer = generated_text[len(input_str):]

    return answer

In [ ]:
%%time

cnt = 0
for doc_idx, document in tqdm(enumerate(unique_documents), total=len(unique_documents)):
    tmp_df = df[df["document"] == document]
    tmp_df.reset_index(drop=True, inplace=True)
    if "B-NAME_STUDENT" in tmp_df["label"].unique().tolist():
        cnt += 1
        if cnt <= 20000:
            idx = 0
            name = []
            name_dict = {}
            while idx < len(tmp_df):
                row = tmp_df.iloc[idx]
                if row["label"] == "B-NAME_STUDENT":
                    start_idx = idx
                    name.append(row["token_str"])

                    # 假设现在这个B就已经是最后了，那么我们需要检测他是否为一个名字
                    if idx == len(tmp_df) - 1:
                        final_name = " ".join(name)
                        name = []
                        if final_name in name_dict:
                            new_df = pd.concat([new_df, tmp_df.loc[idx:idx, ["document", "token", "label"]]])
                            idx += 1
                        else:
                            name_dict[final_name] = 1
                            response = get_response(final_name, document, test)
                            if "yes" in response.lower().strip():
                                new_df = pd.concat([new_df, tmp_df.loc[idx:idx, ["document", "token", "label"]]])
                                idx += 1
                            elif "no" in response.lower().strip():
                                idx += 1
                            else:
                                new_df = pd.concat([new_df, tmp_df.loc[idx:idx, ["document", "token", "label"]]])
                                idx += 1

                    while idx < len(tmp_df) - 1:
                        tmp_idx = idx + 1
                        next_row = tmp_df.iloc[tmp_idx]

                        if next_row["label"] == "I-NAME_STUDENT":
                            name.append(next_row["token_str"])
                            idx = tmp_idx

                            #假设I是最后一行，那么我们也需要直接进行处理
                            if idx == len(tmp_df) - 1:
                                final_name = " ".join(name)
                                name = []
                                if final_name in name_dict:
                                    new_df = pd.concat([new_df, tmp_df.loc[start_idx:, ["document", "token", "label"]]])
                                    idx += 1
                                    break
                                else:
                                    name_dict[final_name] = 1
                                    response = get_response(final_name, document, test)
                                    if "yes" in response.lower().strip():
                                        new_df = pd.concat([new_df, tmp_df.loc[start_idx:, ["document", "token", "label"]]])
                                        idx += 1
                                        break
                                    elif "no" in response.lower().strip():
                                        idx += 1
                                        break
                                    else:
                                        new_df = pd.concat([new_df, tmp_df.loc[start_idx:, ["document", "token", "label"]]])
                                        idx += 1
                                        break

                        else:
                            final_name = " ".join(name)
                            name = []
                            if final_name in name_dict:
                                new_df = pd.concat([new_df, tmp_df.loc[start_idx:idx, ["document", "token", "label"]]])
                                idx = tmp_idx
                                break
                            else:
                                name_dict[final_name] = 1
                                response = get_response(final_name, document, test)
                                if "yes" in response.lower().strip():
                                    new_df = pd.concat([new_df, tmp_df.loc[start_idx:idx, ["document", "token", "label"]]])
                                    idx = tmp_idx
                                    break
                                elif "no" in response.lower().strip():
                                    idx = tmp_idx
                                    break
                                else:
                                    new_df = pd.concat([new_df, tmp_df.loc[start_idx:idx, ["document", "token", "label"]]])
                                    idx = tmp_idx
                                    break
                else:
                    new_df = pd.concat([new_df, tmp_df.loc[idx:idx, ["document", "token", "label"]]])
                    idx += 1
        else:
            new_df = pd.concat([new_df, tmp_df.loc[:, ["document", "token", "label"]]])
    else:
        new_df = pd.concat([new_df, tmp_df.loc[:, ["document", "token", "label"]]])

In [ ]:
new_df.reset_index(drop=True, inplace=True)
row_id = pd.Series(range(len(new_df)), name="row_id")
new_df.insert(0, "row_id", row_id)
new_df.head(50)

In [ ]:
new_df.to_csv("submission.csv", index=False)